In [1]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os
import pandas as pd
import tiktoken
import openai
import numpy as np
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity
from ast import literal_eval
import time
import tiktoken
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

df = pd.read_csv('processed/embeddings.csv', index_col=0)

df=pd.read_csv('processed/embeddings.csv', index_col=0)
df['embeddings'] = df['embeddings'].apply(literal_eval).apply(np.array)


In [4]:
def create_context(
    question, df, max_len=1800, size="ada"
):
    """
    Create a context for a question by finding the most similar context from the dataframe
    """

    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine='text-embedding-ada-002')['data'][0]['embedding']

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')


    returns = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4
        
        # If the context is too long, break
        if cur_len > max_len:
            break
        
        # Else add it to the text that is being returned
        returns.append(row["text"])

    # Return the context
    return "\n\n###\n\n".join(returns)

def answer_question(
    df,
    model="gpt-3.5-turbo-instruct",
    question="How do I import a rule set? If I import from a CSV will anything be overriden?",
    max_len=1800,
    size="ada",
    debug=False,
    max_tokens=150,
    stop_sequence=None
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    context = create_context(
        question,
        df,
        max_len=max_len,
        size=size,
    )
    # If debug, print the raw model response
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        # Create a completions using the questin and context
        response = openai.Completion.create(
            prompt=f"Answer a question about SAS Intelligent Decisioning based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:",
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
            model=model,
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

answer_question(df, question="How do I import a rule set from a JSON file in Intelligent Decisioning?")

'You can import a rule set from a JSON file by opening the rule set you want to import into, clicking on the "Import" button on the Variables tab, selecting "JSON (*.json)" as the file type, and then selecting the JSON file you want to import.'

In [5]:
print(answer_question(df, question="What is a lookup table in Intelligent Decisioning and is that different than a CAS table?"))

A lookup table in Intelligent Decisioning is a table that is used to store and retrieve data for use in published objects. It is different from a CAS table, which is a table stored in SAS Cloud Analytic Services format and used for creating and running tests and scenario tests.


In [7]:
print(answer_question(df, question="What is a Lookupvalue? What purpose does it serve?"))

A Lookupvalue is a function in SAS Intelligent Decisioning that retrieves a lookup value from a lookup table. It is used to retrieve a specific value associated with a lookup key in a lookup table. This function is useful for verifying the existence of a key in a table and retrieving its corresponding value.


In [8]:
print(answer_question(df, question="Can you import an entire decision from JSON or is it only rule sets?"))

It is possible to import an entire decision from JSON using the SAS Intelligent Decisioning CLI. However, the specific command and format for importing decisions from JSON is not provided in the given context.
